In [80]:

import csv 

# Helpers
lower_list = lambda arr: list(map(lambda x: x.lower(), arr))
def generate_html_table(data):
    # Helper function for writing html file
    with open('output.html', 'w') as f:
        # Write the HTML header
        f.write('<!DOCTYPE html>\n<html>\n<head>\n<style>\n')
        # Add some basic CSS for the table
        f.write('table {\nborder-collapse: collapse;\n}\n')
        f.write('table, th, td {\nborder: 1px solid black;\n}\n')
        f.write('th, td {\npadding: 5px;\n}\n')
        f.write('</style>\n</head>\n<body>\n')
        # Write the table header
        f.write('<table>\n<tr>\n<th>ID_string</th>\n<th>ID_ref</th>\n<th>Reference</th>\n<th>html_str</th>\n<th>IsGood</th>\n</tr>\n')
        # Write the table rows
        for row in data:
            f.write('<tr>\n')
            for i in range(len(row)):
                f.write('<td>{}</td>\n'.format(row[i]))
            # Add a checkbox column
            f.write('<td><input type="checkbox"></td>\n')
            f.write('</tr>\n')
        f.write('</table>\n')
        # Write the HTML footer
        f.write('</body>\n</html>')


def read_files(stringfile, reffile, corpfile):
    # Reads input files
    corp, strings, refs = {}, [], []
    with open(stringfile) as main_file, open(reffile) as reference_file, open(corpfile) as corpus_file:
        main_reader = csv.reader(main_file)
        reference_reader = csv.reader(reference_file)
        corpus_reader = csv.reader(corpus_file)

        next(corpus_reader)
        for row in corpus_reader:
            corp_id = row[0]
            word = row[1]
            weight = float(row[2])
            corp[word.lower()] = weight 

        next(main_reader)
        for row in main_reader:
            string_id = row[0]
            string = row[1]
            strings.append((string_id, string))


        next(reference_reader)
        for row in reference_reader:
            ref_id = row[0]
            ref = row[1]
            refs.append((ref_id, ref))
    return corp, strings, refs

#corp, strings, refs = read_files('./data/testmain.csv', './data/testreference.csv', './data/Corpus.csv')
corp, strings, refs = read_files('./data/Main.csv', './data/Reference.csv', './data/Corpus.csv')

In [81]:
# Solution 

csvrows = []
htmlrows = []

from collections import defaultdict
for (string_id, string) in strings:
    for (ref_id, ref) in refs:
        strin_orig_words = (string.replace('-', ' ').split())
        stringwords = lower_list(strin_orig_words)
        
        refwords = (ref.replace('-', ' ').split())
        refwords = lower_list(refwords)
        
        positions = defaultdict(list)     
        

        
        # check if ordered match is possible 
        orderflag = True
        lastrefpos = 0
        for refword in refwords:
            if foundidx >= len(stringwords):
                break
            if refword not in stringwords[lastrefpos:]:
                orderflag = False
            else:
                foundidx = stringwords[lastrefpos:].index(refword) 
                lastrefpos = lastrefpos + foundidx
                positions[refword].append(lastrefpos)
      
        for (i, stringword) in enumerate(stringwords):
            if stringword in refwords:
                if i in positions[stringword]:
                    continue
                positions[stringword].append(i)
        #print (positions) 
        
       # Reverse priorities
        for refword in refwords:
            positions[refword] = list(reversed(positions[refword]))
  
                
        weighted_sum = 0
        match_sum = 0
        
        match = []
        tobold = []
        #orderflag = True
        #lastpos = -1
        for i, word in enumerate(refwords):
            # Check position in reference 
            if len(positions[word]) > 0:
                match.append(word)
                pos = positions[word][-1] # Next position of that word encountered
                tobold.append(pos)
                #if pos < lastpos:
                #    orderflag = False
                #lastpos = pos
             
                positions[word].pop()
                
                weight = 1 if not word in corp else corp[word]
                weighted_sum += weight * len(word)
                match_sum += len(word)
        
        match_percent = match_sum / sum(len(x) for x in refwords)

        if len(match) == len(refwords):
            match_percent_weighted = 1
        else:
             match_percent_weighted = weighted_sum / sum(len(x) for x in refwords)
        
        htmlstr = ''
        for (i, origword) in enumerate(strin_orig_words):
            if i in tobold:
                htmlstr += '<b>'+ origword + '</b>' +' '
            else: 
                htmlstr += origword + ' '      
        
        print('aaa', len(match))
        if len(match) <= 1:
            ordermsg = 'N/A'
        elif orderflag:
            ordermsg = 'TRUE'
        else:
            ordermsg = 'FALSE'
        
        
        if (len(match) >= 1):
            print (string, '|', ref, '|', match, '|', match_percent, match_percent_weighted, ordermsg)
            csvrows.append((string_id, ref_id, ', '.join(match), match_percent, match_percent_weighted, ordermsg))
            htmlrows.append((string_id, ref_id, ref, htmlstr))
        
        
# write csv file
with open('output.csv', mode='w', newline='') as csv_file:
    
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    
    # Write headers to the file
    csv_writer.writerow(['ID_string', 'ID_ref', 'Match', 'Match_Percent', 'Match_Percent_Weighted', 'Match_Order_Same'])
    
    for row in csvrows:
        csv_writer.writerow(row)
generate_html_table(htmlrows)
    

aaa 2
This is an Example | This is | ['this', 'is'] | 1.0 1 TRUE
aaa 0
aaa 1
Bostan is far | This is | ['is'] | 0.3333333333333333 0.03333333333333333 N/A
aaa 0
aaa 2
Ryan is calling this green MX-89 | This is | ['this', 'is'] | 1.0 1 FALSE
aaa 2
Ryan is calling this green MX-89 | MX-89 | ['mx', '89'] | 1.0 1 TRUE
